In [89]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Masking
from tensorflow.keras import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense, GRU,Bidirectional
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb
import numpy as np
tf.__version__
from platform import python_version
np.version.version
python_version()

'3.7.7'

# Variable length issue

In [69]:
(x_train,y_train),(x_test,y_test) = imdb.load_data()

/Users/parzival2108/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/datasets/imdb.py:129: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/Users/parzival2108/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/datasets/imdb.py:130: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [70]:
print(x_train.shape, y_train.shape, x_test.shape,y_test.shape)

(25000,) (25000,) (25000,) (25000,)


#  Masking layer to ignore padded values

In [71]:
x_train = pad_sequences(x_train,maxlen = 234)

In [72]:
x_test = pad_sequences(x_test, maxlen = 234)

In [64]:
# masking_layer = Masking(mask_value=0)
# x_train = x_train[...,np.newaxis]
# x_train_masked = masking_layer(x_train)




In [55]:
# masking_layer = Masking(mask_value=0)
# x_test = x_test[...,np.newaxis]
# x_test_masked = masking_layer(x_test)

In [73]:
print(np.max(x_train), np.max(x_test))

88586 88584


In [74]:
max_index= np.max(x_train_masked)

In [75]:
max_index

88586

# Make the model

In [80]:
model_1 = Sequential([
    Embedding(input_dim=max_index+1, output_dim= 30, input_length=234,mask_zero=True),
    LSTM(units= 128, activation="tanh"),
    Dense(units=1, activation= "sigmoid")
])

In [81]:
model_1.compile(optimizer = "adam",
             loss= "binary_crossentropy", 
             metrics = ["accuracy"])

In [82]:
history_1 = model_1.fit(x_train,y_train, epochs = 3, batch_size = 128)

Train on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 85s 3ms/sample - loss: 0.5400 - accuracy: 0.7176
Epoch 2/3
25000/25000 [==============================] - 80s 3ms/sample - loss: 0.2757 - accuracy: 0.8944
Epoch 3/3
25000/25000 [==============================] - 76s 3ms/sample - loss: 0.1448 - accuracy: 0.9492


# Train with GRU

In [85]:
model_2 = Sequential([
    Embedding(input_dim=max_index+1, output_dim= 30, input_length=234,mask_zero=True),
    GRU(units= 128, activation="tanh"),
    Dense(units=1, activation= "sigmoid")
])

In [86]:
model_2.compile(optimizer = "adam",
             loss= "binary_crossentropy", 
             metrics = ["accuracy"])

In [87]:
history_2 = model_2.fit(x_train,y_train, epochs = 3, batch_size = 128)

Train on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 75s 3ms/sample - loss: 0.5334 - accuracy: 0.7276
Epoch 2/3
25000/25000 [==============================] - 71s 3ms/sample - loss: 0.2308 - accuracy: 0.9103
Epoch 3/3
25000/25000 [==============================] - 69s 3ms/sample - loss: 0.1121 - accuracy: 0.9610


# Model 3

In [91]:
model_3 = Sequential([
    Embedding(input_dim=max_index+1, output_dim= 30, input_length=234,mask_zero=True),
    LSTM(units= 128, return_sequences= True),
    LSTM(units= 128, return_sequences= False),# an other return recurrent layer, return sequence false to only get the final output of the layer
    Dense(units=1, activation= "sigmoid")
])

In [92]:
model_3.compile(optimizer = "adam",
             loss= "binary_crossentropy", 
             metrics = ["accuracy"])

In [93]:
history_3 = model_3.fit(x_train,y_train, epochs = 3, batch_size = 128)

Train on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 171s 7ms/sample - loss: 0.4203 - accuracy: 0.7961
Epoch 2/3
25000/25000 [==============================] - 175s 7ms/sample - loss: 0.1688 - accuracy: 0.9397
Epoch 3/3
25000/25000 [==============================] - 166s 7ms/sample - loss: 0.0867 - accuracy: 0.9726


# Model 4

### Bidirectional wraper

In [95]:
model_4 = Sequential([
    Embedding(input_dim=max_index+1, output_dim= 30, input_length=234,mask_zero=True),
    Bidirectional(layer=LSTM(units= 128, return_sequences= True), merge_mode= "sum",
                  backward_layer=GRU(units=128, go_backwards=True, return_sequences= True)),
    GRU(units=128, return_sequences=False),
    Dense(units=1, activation= "sigmoid")
    
    
])

In [96]:
model_4.compile(optimizer = "adam",
             loss= "binary_crossentropy", 
             metrics = ["accuracy"])

In [97]:
history_4 = model_4.fit(x_train,y_train, epochs = 3, batch_size = 128)

Train on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 205s 8ms/sample - loss: 0.5491 - accuracy: 0.7156
Epoch 2/3
25000/25000 [==============================] - 201s 8ms/sample - loss: 0.2518 - accuracy: 0.9011
Epoch 3/3
25000/25000 [==============================] - 201s 8ms/sample - loss: 0.1157 - accuracy: 0.9604


# Model 5

### Stacked and Bidirectional Recurrent Layers

In [98]:
model_5 = Sequential([
    Embedding(input_dim=max_index+1, output_dim= 30, input_length=234,mask_zero=True),
    Bidirectional(layer=LSTM(units= 128, return_sequences= True), merge_mode= "concat"),
    GRU(units=128, return_sequences=False),
    Dense(units=1, activation= "sigmoid")
])

In [99]:
model_5.compile(optimizer = "adam",
             loss= "binary_crossentropy", 
             metrics = ["accuracy"])

In [100]:
history_5 = model_5.fit(x_train,y_train, epochs = 3, batch_size = 128)

Train on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 220s 9ms/sample - loss: 0.4677 - accuracy: 0.7610
Epoch 2/3
25000/25000 [==============================] - 215s 9ms/sample - loss: 0.1886 - accuracy: 0.9287
Epoch 3/3
25000/25000 [==============================] - 216s 9ms/sample - loss: 0.0787 - accuracy: 0.9733
